## Agile Hardware Design
***
# Hello Chisel

<img src="../resource/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Spring25/)

## Play With Today's Slides

### [https://github.com/agile-hw/lectures](https://github.com/agile-hw/lectures)
* Can work with it directly in browser via [Binder](https://mybinder.org/v2/gh/agile-hw/lectures/HEAD?urlpath=tree)
* Can clone and run Jupyter locally
  * Will need to install additional things for presentation, Scala, and Chisel
  * Repo contains `install.sh` to help with this

## Plan for Today

* Brief taste of Scala
* Brief taste of Chisel
* _Close the loop:_ build & test simple Chisel module

## Scala Language Summary

<img src="../01-intro/images/scala-spiral.png" alt="scala logo" style="float:right; width: 80px;"/>

_**Language Features**_
* Object oriented with strong static type system
* Native support for functional programing
* Runs on top of the JVM (and can interoperate with Java binaries)
* Tries to catch many potential errors at compile time

_**Rationale for Using Scala**_
* Great support for implementing embedded domain-specific languages (DSL), e.g. Chisel
* Object oriented and functional features help make great _generators_
* Type system and included standard library’s collections

## Scala Execution Mechanisms

_**Standard Compilation -> Execution**_
* Compile Scala program (as Java bytecode) and run on JVM
* Code needs to be structured in classes and have a `main`
* Typically use appropriate build tool (e.g. sbt) or an IDE (e.g. IntelliJ)

_**Read-Eval-Print Loop (REPL)**_
* Can launch a REPL (interpreter) and write (then evaluate) a single line at a time
* Great for testing out features
* The Jupyter notebooks in this course (where we can execute snippets in isolation) are built on an extension to this ([Almond](https://almond.sh))

## Scala Literals

* Common Simple Scala Types - `Int`, `Float`, `Long`, `Double`, `Byte`, `Char`, `String`
* _Syntax Note:_ semicolons are optional and rarely used

In [ ]:
2 + 3
5.0 / 2
"hello"

## Scala Type Inference

* Everything is an object, even simple types
* Types must be known/resolved at compile time (_statically typed_)
* Scala has _type inference_, so can often omit type specifier
* _Syntax Note:_ unlike C/Java, type goes after name (instead of before)

In [ ]:
4
4: Int
4: Float
4: Double
4: Char
4.toFloat

## Declaring Scala Variables

### `var` - **Mutable** variable (_discouraged_)
* Can reassign, like conventional languages

### `val` - **Immutable** variable (_encouraged_)
* Enables write-once semantics common to many functional languages
* Allows compiler to safely perform more ambitious optimizations
* Can increase code clarity by renaming values each step of the way

In [ ]:
var mutX = 0
mutX = 2
val constX = 42

## Chisel Is Embedded in Scala

* Chisel designs are Scala programs (i.e. everything we write in this course is Scala)
* A Chisel design is simply a Scala program that makes use of the Chisel library
* Thanks to Scala language features, using Chisel library operations often feels like a full-fledged language
  * Operator overloading and concise syntax

## Chisel Tool Flow (Frontend)

<img src="images/frontend.svg" alt="Chisel frontend" style="width:80%;margin:auto"/>

* The generated Circuit (`.fir` file) is a specific design instance, and it can be passed off to a _backend_ for simulation or implementation

## Loading The Chisel Library Into a Notebook

In [ ]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Simple Chisel Types

* Scala literals must be cast to Chisel

### `Bool` - single-bit logic signal (`.B`)
* Do not confuse with Scala's `Boolean`

### `UInt` - unsigned integer (`.U`)
* Bitwidth set explicitly or inferred

### `SInt` - signed integer (`.S`)
* Operates like `UInt` but signed


In [ ]:
0.B
true
true.B
val myBool: Bool = true.B

6
6.U
6.U(8.W)
val myUInt: UInt = 4.U
val myUInt8 = 4.U(8.W)

-2
-2.S

## Chisel Operators (Brief Intro)

* Most of the operators you would expect exist
    * Some of them have different symbols (e.g. `===` for equality test)
    * Pay attention to result widths
    * Consult [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/download/3.6.0/chisel_cheatsheet.pdf) for brief summary
* _**Logical:**_ `!`, `&&`, `||`
* _**Arithmetic:**_ `+`, `-`, `*`, `/`, `%`
* _**Bitwise:**_ `~`, `&`, `|`, `^`
* _**Relational:**_ `===`, `=/=`, `<`, `<=`, `>`, `>=`
* _**Shifts:**_ `<<`, `>>`
* _**Others:**_ extraction, fill, concatenation, mux, reductions


In [ ]:
val a = 7.U
val b = 5.U

// Not expected to work (needs to be inside Module)
// a + b

## First Chisel Module

In [ ]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    io.c := io.a ^ io.b
}

<img src="images/myxor.svg" alt="MyXOR schematic"  style="width:80%;margin:auto"/>

## Looking At Generated Design

In [ ]:
printVerilog(new MyXOR)

In [ ]:
visualize(() => new MyXOR)

## Chisel Tool Flow (Backend)
<p><p>

<img src="images/backend.svg" alt="Chisel backend" style="width:75%;margin:auto"/>

## Brief ChiselTest Intro

* Can write a Scala program to interact with simulation of elaborated design
* Can set inputs and look at outputs
* Can use full power of Scala to generate test inputs and outputs to compare with
* Will continue to cover more features in coming lectures

### `poke` - set value of wire

### `peek` - read value of wire

### `expect` - read value and compare (assert)

## ChiselTest Execution

In [ ]:
test(new MyXOR()) { x =>
    x.io.a.poke(0.B)
    x.io.b.poke(0.B)
    x.io.c.expect(0.B)  // 0 ^ 0

    x.io.a.poke(0.B)
    x.io.b.poke(1.B)
    x.io.c.expect(1.B)  // 0 ^ 1

    x.io.a.poke(1.B)
    x.io.b.poke(0.B)
    x.io.c.expect(1.B)  // 1 ^ 0

    x.io.a.poke(1.B)
    x.io.b.poke(1.B)
    x.io.c.expect(0.B)  // 1 ^ 1
}

## Chisel Versioning

<img src="../01-intro/images/chisel_logo.svg" alt="chisel logo" style="float:right"/>

* This quarter, we will be using **Chisel 3.6.1** for lecture, labs, and homework
  * You are welcome to use a newer Chisel for the project
  * _Rationale:_ Chisel is undergoing big changes, and not quite stabilized
* Provided code and environments will correctly pull in the right version
  * When reading documentation, be aware may be describing a feature in newer version
* Relevant Chisel versions
  * 3.6.1 (June 2024) is the bridge from the old to the new
  * 6.7 (March 2025) is the most recent released version
  * 7 is under public development
  * There is no version 4, as it was skipped to make version jump apparent